In [1]:
import xlrd
import math as m

# classe con le caratteristiche di ogni città
class nodo():
	def __init__(self,ide,nome,lat,lon,stato,identificativo,defi,pot,coll,adia): 
		self.ide=ide
		self.lat=lat
		self.lon=lon
		self.nome=nome
		self.stato=stato
		self.identificativo=identificativo
		self.defi=defi
		self.pot=pot
		self.coll=coll
		self.adia=adia

# calcolo della distanza fra due punti
def dist2punti(latA,lonA,latB,lonB):
	Rt=6378.388
	lat_A = (m.pi*latA)/180.
	lat_B = (m.pi*latB)/180.
	lon_A = (m.pi*lonA)/180.
	lon_B = (m.pi*lonB)/180.
	dl=abs(lon_A-lon_B)
	#print(lat_A,lon_A,lat_B,lon_B)
	p = m.acos((m.sin(lat_B)*m.sin(lat_A))+(m.cos(lat_B)*m.cos(lat_A)*m.cos(dl)))
	if(lon_A>0 and lon_B>0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A<0 and lon_B<0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A>0 and lon_B<0):
		if((lon_A+abs(lon_B))<m.pi): 
			p=float('inf')
		else:
			p=p
	if(lon_A<0 and lon_B>0):
		if((lon_B+abs(lon_A))<m.pi): 
			p=p
		else:
			p=float('inf')
	return(p*Rt) 

#Lettura dei dati dal file
dati=xlrd.open_workbook('/Users/Tonino222power/Desktop/City2.xlsx')
foglio = dati.sheet_by_name('Foglio1')
nR=foglio.nrows
nC=foglio.ncols

#Imposto la citta di inizio
citta_inizio="London"
inizio="1826645935"



nome=[]
lat=[]
log=[]
stato=[]
stato2=[]
pop=[]
iden=[]
pesi=[]
nodi=[]

#Inserisco i dati del file in vettori per i futuri calcoli
for i in range(1,nR):	
	nome.append(foglio.cell(i,0).value)
	lat.append(foglio.cell(i,2).value)
	log.append(foglio.cell(i,3).value)
	stato.append(foglio.cell(i,6).value)
	stato2.append(foglio.cell(i,7).value)
	pop.append(foglio.cell(i,9).value)
	iden.append(str(int(foglio.cell(i,10).value)))

print("Lettura dati da file terminata")

# controllo che le città abbiano una popolazione altrimenti metto a zero
for i in range(0,nR-1):
	if(pop[i]==''):
		pop[i]=0
		#print(i,pop[i],iden[i])
print("Controllo popolazioni terminato")


#controllo che non ci siano doppioni delle città in caso le elimino dal calcolo
elim=[]
elim1=[]

for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(lat[j]==lat[i] and log[j]==log[i] and iden[j]!=iden[i]):
			print(j)
			elim.append(j)

for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(iden[j]==iden[i] and lat[j]!=lat[i] and log[j]!=log[i]):
			print(k)
			elim1.appen(j)

for j in elim:
	print("rimosso per coordinate coincidenti: ",iden[j])
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)

for j in elim1:
	print("rimosso per identificativo coincidente: ",iden[j])
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)

#controllo la nuova lunghezza delle liste dopo aver eliminato i doppioni
print(len(nome))
nR=len(nome)
#imposto la variabile citta finale con le stesse caratterstiche di quella iniziale
#solo una questione di comodita
citta_fine=citta_inizio+"_fine"
fine="1111111111"

for i in range(0,nR):
	if(iden[i]==inizio):
		nome.append(citta_fine)
		lat.append(lat[i])
		log.append(log[i]-0.1)
		stato.append(stato[i])
		stato2.append(stato2[i])
		pop.append(pop[i])
		iden.append(fine)

#Aggiorno la lunghezza delle liste
print("Numerosità città")
print(len(nome))
nN=len(nome)

#ciclo che calcola le distanze fra i punti e prende in considerazione solo le tre piu vicine ad ognuna
for i in range(0,nN):
	vicini=[]
	for j in range(0,nN):
		if(iden[i]==inizio and iden[j] == fine):
			continue
		if(iden[j]==inizio):
			continue
		if(iden[i]!=iden[j]):
			d=dist2punti(lat[i],log[i],lat[j],log[j])
			if(len(vicini)==0):
				vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
			else:	
				for k in range(0,len(vicini)):
					if(d<vicini[k][1]):
						vicini.insert(k,[iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
						if(len(vicini)>3):					
							vicini.pop(3)
						break
					if(k==2 and len(vicini)<3):
						vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
	#inserisco le tre piu vicine in un dizionario con chiave la citta di riferimento
	#e valori le tre citta piu vicine con i relativi pesi
	diz={}
	for l in range(0,len(vicini)):
		# peso per la distanza 2 alla piu vicina 4 a quella in mezzo e 8 alla piu distante
		pes=2**(l+1)
		if(vicini[l][2]>200000):
			pes=pes+2
		# 2 punti in piu se la citta ha piu di 200 000 abitanti
		if(vicini[l][3] != vicini[l][4]):
			pes=pes+2		
		diz[vicini[l][0]]=pes 
		# due punti in piu se la citta e in un altro stato
	print("Ancora "+str(nN-i)+" citta' da controllare" )
	# lista che contine tutti i dizionari
	pesi.append(diz)
	# aggiungo alla lista ogni citta con caratteristiche e dizionario con i pesi
	nodi.append(nodo(i,nome[i],lat[i],log[i],stato[i]+"-"+stato2[i],iden[i],True,float('inf'),"",pesi[i])) 

print("Algoritmo di calcolo del grafo utilizzando l'algoritmo Dijkstra")
# per capire il funzionamento dell'algoritmo è stata consultata la pagina di Wikipedia

# imposto la citta di partenza come controllata
for x in nodi:
	if(x.identificativo == inizio):
		x.defi=False
		x.pot=0
		x.coll=inizio
		i=x.ide
k=0
while True:
	for x in nodi[i].adia.keys():
		for y in nodi:
			if (y.identificativo == x and y.defi ):
				somma=nodi[i].pot + nodi[i].adia[x]
				if(y.pot>somma):
					y.pot=somma
					y.coll=nodi[i].identificativo
	m=float('inf')
	for z in nodi:
		if(z.defi and z.pot<m):
			m=z.pot
			n=z.ide
	
	nodi[n].defi=False
	i=n
	print(nodi[i].nome)
	# se trova la citta finale  esce e salva il peso come ore di percorso
	if(nodi[i].identificativo == fine ):
		lung=nodi[i].pot
		break
	k=k+1

percorso=[]
percorso_nomi=[]

# creo a ritroso il percoro partendo dalla fine  arrivando alla partenza
s=fine
while True:
	percorso.append(s)
	for x in nodi:
		if(x.identificativo == s):
			s=x.coll
	if(s==inizio):
		percorso.append(s)
		break

# passo da percorso che contine gli identificativi alla stessa lista con i nome relativi delle citta
for x in percorso:
	for y in nodi:
		if(y.identificativo==x):
			percorso_nomi.append(y.nome+" ("+y.stato+")")

# creo una lista per stampare i nomi del percorso
strada=""
for i in range(0,len(percorso_nomi)-1):
	strada=strada+percorso_nomi[len(percorso_nomi)-1-i]+" - "
strada=strada+" "+percorso_nomi[0]

print("-------------------Risultato----------------------------")
print("Percorso terminato in "+str(lung)+" ore o "+str(float(lung)/24.)+" giorni. (tempo minimo)" )
print("Tragitto: " + strada)
print("--------------------------------------------------------")

Lettura dati da file terminata
Controllo popolazioni terminato
3914
2943
3977
1155
183
1426
rimosso per coordinate coincidenti:  1484228590
rimosso per coordinate coincidenti:  1484980825
rimosso per coordinate coincidenti:  1418863956
rimosso per coordinate coincidenti:  1484504936
rimosso per coordinate coincidenti:  1484559591
rimosso per coordinate coincidenti:  1076190259
4994
Numerosità città
4995
Ancora 4995 citta' da controllare
Ancora 4994 citta' da controllare
Ancora 4993 citta' da controllare
Ancora 4992 citta' da controllare
Ancora 4991 citta' da controllare
Ancora 4990 citta' da controllare
Ancora 4989 citta' da controllare
Ancora 4988 citta' da controllare
Ancora 4987 citta' da controllare
Ancora 4986 citta' da controllare
Ancora 4985 citta' da controllare
Ancora 4984 citta' da controllare
Ancora 4983 citta' da controllare
Ancora 4982 citta' da controllare
Ancora 4981 citta' da controllare
Ancora 4980 citta' da controllare
Ancora 4979 citta' da controllare
Ancora 4978 cit

Ancora 4766 citta' da controllare
Ancora 4765 citta' da controllare
Ancora 4764 citta' da controllare
Ancora 4763 citta' da controllare
Ancora 4762 citta' da controllare
Ancora 4761 citta' da controllare
Ancora 4760 citta' da controllare
Ancora 4759 citta' da controllare
Ancora 4758 citta' da controllare
Ancora 4757 citta' da controllare
Ancora 4756 citta' da controllare
Ancora 4755 citta' da controllare
Ancora 4754 citta' da controllare
Ancora 4753 citta' da controllare
Ancora 4752 citta' da controllare
Ancora 4751 citta' da controllare
Ancora 4750 citta' da controllare
Ancora 4749 citta' da controllare
Ancora 4748 citta' da controllare
Ancora 4747 citta' da controllare
Ancora 4746 citta' da controllare
Ancora 4745 citta' da controllare
Ancora 4744 citta' da controllare
Ancora 4743 citta' da controllare
Ancora 4742 citta' da controllare
Ancora 4741 citta' da controllare
Ancora 4740 citta' da controllare
Ancora 4739 citta' da controllare
Ancora 4738 citta' da controllare
Ancora 4737 ci

Ancora 4525 citta' da controllare
Ancora 4524 citta' da controllare
Ancora 4523 citta' da controllare
Ancora 4522 citta' da controllare
Ancora 4521 citta' da controllare
Ancora 4520 citta' da controllare
Ancora 4519 citta' da controllare
Ancora 4518 citta' da controllare
Ancora 4517 citta' da controllare
Ancora 4516 citta' da controllare
Ancora 4515 citta' da controllare
Ancora 4514 citta' da controllare
Ancora 4513 citta' da controllare
Ancora 4512 citta' da controllare
Ancora 4511 citta' da controllare
Ancora 4510 citta' da controllare
Ancora 4509 citta' da controllare
Ancora 4508 citta' da controllare
Ancora 4507 citta' da controllare
Ancora 4506 citta' da controllare
Ancora 4505 citta' da controllare
Ancora 4504 citta' da controllare
Ancora 4503 citta' da controllare
Ancora 4502 citta' da controllare
Ancora 4501 citta' da controllare
Ancora 4500 citta' da controllare
Ancora 4499 citta' da controllare
Ancora 4498 citta' da controllare
Ancora 4497 citta' da controllare
Ancora 4496 ci

Ancora 4277 citta' da controllare
Ancora 4276 citta' da controllare
Ancora 4275 citta' da controllare
Ancora 4274 citta' da controllare
Ancora 4273 citta' da controllare
Ancora 4272 citta' da controllare
Ancora 4271 citta' da controllare
Ancora 4270 citta' da controllare
Ancora 4269 citta' da controllare
Ancora 4268 citta' da controllare
Ancora 4267 citta' da controllare
Ancora 4266 citta' da controllare
Ancora 4265 citta' da controllare
Ancora 4264 citta' da controllare
Ancora 4263 citta' da controllare
Ancora 4262 citta' da controllare
Ancora 4261 citta' da controllare
Ancora 4260 citta' da controllare
Ancora 4259 citta' da controllare
Ancora 4258 citta' da controllare
Ancora 4257 citta' da controllare
Ancora 4256 citta' da controllare
Ancora 4255 citta' da controllare
Ancora 4254 citta' da controllare
Ancora 4253 citta' da controllare
Ancora 4252 citta' da controllare
Ancora 4251 citta' da controllare
Ancora 4250 citta' da controllare
Ancora 4249 citta' da controllare
Ancora 4248 ci

Ancora 4027 citta' da controllare
Ancora 4026 citta' da controllare
Ancora 4025 citta' da controllare
Ancora 4024 citta' da controllare
Ancora 4023 citta' da controllare
Ancora 4022 citta' da controllare
Ancora 4021 citta' da controllare
Ancora 4020 citta' da controllare
Ancora 4019 citta' da controllare
Ancora 4018 citta' da controllare
Ancora 4017 citta' da controllare
Ancora 4016 citta' da controllare
Ancora 4015 citta' da controllare
Ancora 4014 citta' da controllare
Ancora 4013 citta' da controllare
Ancora 4012 citta' da controllare
Ancora 4011 citta' da controllare
Ancora 4010 citta' da controllare
Ancora 4009 citta' da controllare
Ancora 4008 citta' da controllare
Ancora 4007 citta' da controllare
Ancora 4006 citta' da controllare
Ancora 4005 citta' da controllare
Ancora 4004 citta' da controllare
Ancora 4003 citta' da controllare
Ancora 4002 citta' da controllare
Ancora 4001 citta' da controllare
Ancora 4000 citta' da controllare
Ancora 3999 citta' da controllare
Ancora 3998 ci

...

Ancora 328 citta' da controllare
Ancora 327 citta' da controllare
Ancora 326 citta' da controllare
Ancora 325 citta' da controllare
Ancora 324 citta' da controllare
Ancora 323 citta' da controllare
Ancora 322 citta' da controllare
Ancora 321 citta' da controllare
Ancora 320 citta' da controllare
Ancora 319 citta' da controllare
Ancora 318 citta' da controllare
Ancora 317 citta' da controllare
Ancora 316 citta' da controllare
Ancora 315 citta' da controllare
Ancora 314 citta' da controllare
Ancora 313 citta' da controllare
Ancora 312 citta' da controllare
Ancora 311 citta' da controllare
Ancora 310 citta' da controllare
Ancora 309 citta' da controllare
Ancora 308 citta' da controllare
Ancora 307 citta' da controllare
Ancora 306 citta' da controllare
Ancora 305 citta' da controllare
Ancora 304 citta' da controllare
Ancora 303 citta' da controllare
Ancora 302 citta' da controllare
Ancora 301 citta' da controllare
Ancora 300 citta' da controllare
Ancora 299 citta' da controllare
Ancora 298

Ancora 76 citta' da controllare
Ancora 75 citta' da controllare
Ancora 74 citta' da controllare
Ancora 73 citta' da controllare
Ancora 72 citta' da controllare
Ancora 71 citta' da controllare
Ancora 70 citta' da controllare
Ancora 69 citta' da controllare
Ancora 68 citta' da controllare
Ancora 67 citta' da controllare
Ancora 66 citta' da controllare
Ancora 65 citta' da controllare
Ancora 64 citta' da controllare
Ancora 63 citta' da controllare
Ancora 62 citta' da controllare
Ancora 61 citta' da controllare
Ancora 60 citta' da controllare
Ancora 59 citta' da controllare
Ancora 58 citta' da controllare
Ancora 57 citta' da controllare
Ancora 56 citta' da controllare
Ancora 55 citta' da controllare
Ancora 54 citta' da controllare
Ancora 53 citta' da controllare
Ancora 52 citta' da controllare
Ancora 51 citta' da controllare
Ancora 50 citta' da controllare
Ancora 49 citta' da controllare
Ancora 48 citta' da controllare
Ancora 47 citta' da controllare
Ancora 46 citta' da controllare
Ancora 4

Derbent
Yozgat
Ulan-Ude
Tokat
Darhan
Ürümqi
Samsun
Gorgān
Niğde
Bojnūrd
Fukang
Nevşehir
Rustavi
Mersin
Sumqayıt
Aksaray
Daşoguz
Urganch
Ünye
Xırdalan
Ulaanbaatar
Turpan
Sivas
Chita
Gəncə
Sabzevār
Ordu
Shāhrūd
Altay
Kayseri
Ashgabat
Nicosia
Tarsus
Manzhouli
Talas
Qūchān
Mingəçevir
Baku
Türkmenabat
Kumul
Neyshābūr
Giresun
Adana
Yakeshi
Bukhara
Dunhuang
Kozan
Genhe
Mashhad
Trabzon
Torbat-e Ḩeydarīyeh
Kadirli
Kogon Shahri
Kāshmar
Latakia
Zalantun
Yingmen
Erzincan
Ahar
Torbat-e Jām
Osmaniye
Jiayuguan
Rize
Ţarţūs
Nehe
Kahramanmaraş
Qarshi
Bingöl
Batumi
Samandağ
Longjiang
Xibeijie
Samarkand
Ardabīl
Herāt
Mary
Shahrisabz
Zhangye
Homs
Zanjān
Hatay
Wudalianchi
Hong’an
Muş
Kutaisi
Kilis
Qiqihar
Gaziantep
Erzurum
Beian
Ḩamāh
İskenderun
Maīmanah
Jizzax
Kars
Farāh
Rasht
Jinchang
Batman
Siirt
Nizip
As Salamīyah
Daqing
Wuwei
Hailun
Denov
Lashkar Gāh
Khŭjand
Shibirghān
Idlib
Patnos
Gyumri
Aleppo
Tieli
Qazvīn
Ağrı
Iğdır
Olmaliq
Şırnak
Suihua
Şanlıurfa
Baiyin
Heihe
Dushanbe
Anda
Mazār-e Sharīf
Adıyaman
C

Puerto Vallarta
Medinīpur
Chittoor
Saharsa
Pomona
Jāmuria
Torrance
Bhīmavaram
Tepatitlán de Morelos
Tādepallegūdem
Ciudad Lázaro Cárdenas
South Gate
Hawthorne
Chennai
Ganzhou
Hefei
Chuzhou
Anshan
Fushun
Shuyangzha
Jianguang
Zapopan
Yingcheng
Torreón
Āsansol
Provo
Pencheng
Jitpur
Guadalupe
Regina
Ontario
Pālakollu
Ciudad Hidalgo
San Francisco del Rincón
Downey
Janakpur
Bellflower
Nanchang
Jiujiang
Heyuan
Tieling
Benxi
Salt Lake City
Kathmandu
Laiyang
Saltillo
Wenchang
Luofeng
Bhāgalpur
Anaheim
Barddhamān
Diaobingshancun
Uluberiya
Vellore
Rancho Cucamonga
Grand Junction
Fullerton
Navadwīp
Billings
Zihuatanejo
Lakewood
Fuzhou
Liaoyang
León de los Aldama
Winnipeg
Aguascalientes
Chaohucun
Ruiming
Zhuangyuan
Tianchang
Kaiyuan
Long Beach
Irapuato
Rājahmundry
Purnea
Puducherry
Haldia
Cuddalore
Krishnanagar
Silao
Orange
Boulder
East Los Angeles
Jurupa Valley
Norwalk
San Felipe del Progreso
Bismarck
Whittier
Frontera
Dongguan
Nanjing
Anqing
Ma’anshan
Huaiyin
Hāora
Xingcheng
Zhuanghe
Acapulco de 

Hōfu
Columbia
Silver Spring
Zunyi
Jacksonville
Buffalo
Qingzhen
Lancaster
Markham
Vaughan
Việt Trì
Savannah
Chesapeake
Puerto Cortés
Sơn Tây
Iwakuni
Savannakhet
Ajax
Nakhon Pathom
Pine Hills
Guiyang
Baltimore
Virginia Beach
Raleigh
Kumamoto
Columbia
Reading
Tegucigalpita
Richmond Hill
Siem Reap
Greenville
Whitby
Amherst
Ahuachapán
Ubon Ratchathani
Bangkok
Hanoi
Havana
Orlando
Matsuyama
Ōita
Palm Coast
Fuquan
Deltona
Oshawa
Matanzas
El Progreso
North Charleston
Hatsukaichi
Pottstown
Salisbury
Phnom Penh
Rochester
Duyun
Kissimmee
Thái Nguyên
Santa Ana
Sonsonate
Đồng Hới
Imabari
La Ceiba
Estelí
Kampong Cham
Beppu
Jacksonville
Quảng Trị
Hiroshima
Allentown
Charleston
Bắc Ninh
Palm Bay
Syracuse
Cienfuegos
Comayagua
Jinotega
Dover
Saijō
Upper Darby
Baicheng
Tongren
Port St. Lucie
Myrtle Beach
Kure
Hà Tĩnh
Binghamton
Tây Ninh
Santa Tecla
Niihama
Utica
Prey Veng
Philadelphia
Hechi
Hải Dương
Huế
Wilmington
Atlantic City
Santa Clara
Higashi-Hiroshima
Matagalpa
West Palm Beach
Comayagüela
San Sal